# Timing analysis of chain CZ gate - Numpy implementation

In [ ]:
# Import packages
import numpy as np
import time
import scipy.linalg
from scipy.sparse import csc_matrix, linalg as sla
from joblib import Parallel, delayed
import csv
import os
import pandas as pd

# Set data type
data_type = np.complex64

## Time-dependent Shrödinger equation solvers

In [ ]:
# Spectral method solver
def spectral_evol(H, psi, tau):

    def exp_mat(H, tau):

        eiglist = np.linalg.eigh(H)
        eigenvalues = eiglist[0]
        eigenvectors = eiglist[1]
        print(eigenvalues)
        eigenvectors_inv = np.linalg.inv(eigenvectors)
        exp_diag = np.diag(np.exp(-1j*eigenvalues*tau))

        return np.dot(eigenvectors,np.dot(exp_diag,eigenvectors_inv))
    
    exp_H = exp_mat(H, tau)
    
    return np.dot(exp_H,psi)

# Crank-Nicolson solver
def crank_nicolson(H, psi, tau, n_iter):
    
    t = np.linspace(0, tau, n_iter)
    dt = t[1] - t[0]
    
    A = np.eye(H.shape[0], dtype=data_type) + 1j * H * dt/2
    B = np.eye(H.shape[0], dtype=data_type) - 1j * H * dt/2

    b = np.dot(B,psi) 

    for index, step in enumerate(t):
        
        psi = np.linalg.solve(A, b)
        b = np.dot(B,psi) 
        
    return psi  

# Crank-Nicolson LU optimized solver
def crank_nicolson_LU(H, psi, tau, n_iter):
    
    t = np.linspace(0, tau, n_iter)
    dt = t[1] - t[0]
    
    A = np.eye(H.shape[0], dtype=data_type) + 1j * H * dt/2
    B = np.eye(H.shape[0], dtype=data_type) - 1j * H * dt/2

    b = np.dot(B,psi) 
    
    L, U = scipy.linalg.lu(A, permute_l = True)

    for index, step in enumerate(t):
        
        y = scipy.linalg.solve_triangular(L,b,lower=True)
        psi = scipy.linalg.solve_triangular(U,y,lower=False)        
        b = np.dot(B,psi) 
    
    return psi

# Crank-Nicolson LU optimized solver for sparse matrix
def crank_nicolson_LU_sparse(H, psi, tau, n_iter):
    
    t = np.linspace(0, tau, n_iter)
    dt = t[1] - t[0]
    
    A = np.eye(H.shape[0], dtype=data_type) + 1j * H * dt/2
    B = np.eye(H.shape[0], dtype=data_type) - 1j * H * dt/2

    b = np.dot(B,psi) 
    
    A = csc_matrix(A)
    lu = sla.splu(A) 

    for index, step in enumerate(t):
        
        psi = lu.solve(b)     
        b = np.dot(B,psi) 
    
    return psi 

## Chain CZ-gate implementation

In [ ]:
# Adjiont operator
def adjoint(psi):
    return np.conj(np.transpose(psi))

# Tensor operator
def tensor(a,b):
    return np.kron(a,b)

In [ ]:
# Computational base vector initialization
def basis(dim, state):
    vect = np.zeros((dim,1),dtype=data_type)
    vect[state] = 1
    return vect

In [ ]:
# Optimal phase between two pulses
def exp_xi(Delta,Omega,tau):
    
    y = Delta/Omega
    s = Omega * tau
    
    a = np.sqrt(y**2+1)
    b = s*a/2

    return (a*np.cos(b) + 1j*y*np.sin(b)) / (-a*np.cos(b) + 1j*y*np.sin(b))

In [ ]:
# Definition of the Hamiltonian for two-qubit CZ gate
def hamiltonian(Omega,Delta):
    
    psi00 = tensor(basis(3,0),basis(3,0))
    psi01 = tensor(basis(3,0),basis(3,1)) 
    psi0r = tensor(basis(3,0),basis(3,2))
    psi10 = tensor(basis(3,1),basis(3,0))
    psi11 = tensor(basis(3,1),basis(3,1)) 
    psi1r = tensor(basis(3,1),basis(3,2))
    psir0 = tensor(basis(3,2),basis(3,0))
    psir1 = tensor(basis(3,2),basis(3,1))
    psirr = tensor(basis(3,2),basis(3,2))    

    H0  = 0 * tensor( adjoint(psi00),psi00)
    
    H01 = 1/2 * ( Omega * tensor( adjoint(psi01),psi0r) + 
             np.conj(Omega) * tensor( adjoint(psi0r),psi01) ) - Delta * tensor( adjoint(psi0r),psi0r)
    
    H10 = 1/2 * ( Omega * tensor( adjoint(psi10),psir0) + 
             np.conj(Omega) * tensor( adjoint(psir0),psi10) ) - Delta * tensor( adjoint(psir0),psir0)

    H2  = 1/2 * ( Omega * ( tensor( adjoint(psi11),psir1) + tensor( adjoint(psi11),psi1r) ) 
            + np.conj(Omega) * ( tensor( adjoint(psir1),psi11) + tensor( adjoint(psi1r),psi11) ) 
        ) - Delta/2 * ( tensor( adjoint(psir1),psir1) + tensor( adjoint(psir1),psi1r) 
                          + tensor( adjoint(psi1r),psir1) + tensor( adjoint(psi1r),psi1r))

    H = H0 + H01 + H10 + H2
    
    if not np.allclose(H,adjoint(H)):
        print("ERROR: Hamiltonian is not hermitian!")
    
    return H

In [ ]:
# Chain state initialization
def chain_init(N,state_first,state_last):
    
    psi = basis(3,state_first) 
    
    for i in range(N-2):
        psi = tensor(psi,basis(3,0))
        
    psi = tensor(psi,basis(3,state_last))
    
    return psi

In [ ]:
# Chain CZ-gate implementation
def chain_CZ_gate_time(psi,Omega,Delta,tau,method='spectral_evol',niter=100):
    
    start = time.time()
        
    N = int(np.log10(psi.shape[0])/(np.log10(3)))  # Number of qubits in the chain
    I = np.eye(3, dtype=data_type) # Define identity matrix
    Hp1 = hamiltonian(Omega,Delta) # First pulse hamiltonian
    Hp2 = hamiltonian(Omega * exp_xi(Delta,Omega,tau), Delta) # Second pulse hamiltonian  
    
    for i in range(N-1):
        
        mat_list1 = [I]*(N-1)
        mat_list2 = [I]*(N-1)
        mat_list1[i] = Hp1
        mat_list2[i] = Hp2
        H1 = mat_list1[0]
        H2 = mat_list2[0]
        
        for j in range(N-2):
            H1 = tensor(H1, mat_list1[j+1])
            H2 = tensor(H2, mat_list2[j+1])
        
        if method == 'spectral_evol':
            psi = spectral_evol(H1, psi, tau)   
            psi = spectral_evol(H2, psi, tau) 
        
        elif method == 'cn':
            psi = crank_nicolson(H1, psi, tau, niter)
            psi = crank_nicolson(H2, psi, tau, niter)
        
        elif method == 'cn_LU':
            psi = crank_nicolson_LU(H1, psi, tau, niter)
            psi = crank_nicolson_LU(H2, psi, tau, niter)
            
        elif method == 'cn_LU_sp':
            psi = crank_nicolson_LU_sparse(H1, psi, tau, niter)
            psi = crank_nicolson_LU_sparse(H2, psi, tau, niter)
            
        else: 
            print("ERROR: no valid input method!")
            return None
    
    end = time.time()
    
    return end-start

## Timing analysis as a function of the number of qubit N

In [ ]:
# Run in parallel chain CZ gate for different N and iterations
def chain_CZ_gate_execution(N, method, niter=100, ntimes=10):

    Omega   = 1
    frac_DO = 0.377371
    prod_Ot = 4.29268
    Delta = frac_DO * Omega
    tau = prod_Ot / Omega

    filename = "numpy_"+str(method)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)
    
    print("Method: ", method)
    for i in range(len(N)):
        print("N: ", N[i])
        
        state_first = 1
        state_last = 1
        psi_init = chain_init(N[i],state_first,state_last)
        
        res = Parallel(n_jobs=-1, verbose=11)(delayed(chain_CZ_gate_time)(psi_init,Omega,Delta,tau,method,niter) for t in range(ntimes))

        with open(filename, "a") as f:
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res), np.std(res)/np.sqrt(ntimes)))

In [ ]:
# Spectral method solver
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution(N, 'spectral_evol', ntimes=10)

In [ ]:
# Crank-Nicolson solver
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution(N, 'cn', 100, ntimes=10)

In [ ]:
# Crank-Nicolson with LU decomposition solver
N = [2, 3, 4, 5, 6, 7, 8]
chain_CZ_gate_execution(N, 'cn_LU', 100, ntimes=10)

In [ ]:
# Crank-Nicolson with LU decomposition with sparse matrix solver
N = [2, 3, 4, 5, 6, 7, 8]
chain_CZ_gate_execution(N, 'cn_LU_sp', 100, ntimes=10)

## Fidelity analysis as a function of the number of iterations

In [ ]:
# Fidelity computation
def fidelity(psi1,psi2):
    fid = np.abs( np.dot(adjoint(psi1),psi2) )**2
    return fid[0][0]

# Chain CZ-gate implementation
def chain_CZ_gate_fid(psi,Omega,Delta,tau,method='spectral_evol',niter=100):
        
    psi_ref = psi        
    N = int(np.log10(psi.shape[0])/(np.log10(3)))  # Number of qubits in the chain
    I = np.eye(3, dtype=data_type) # Define identity matrix
    Hp1 = hamiltonian(Omega,Delta) # First pulse hamiltonian
    Hp2 = hamiltonian(Omega * exp_xi(Delta,Omega,tau), Delta) # Second pulse hamiltonian  
    
    for i in range(N-1):
        
        start = time.time()
        mat_list1 = [I]*(N-1)
        mat_list2 = [I]*(N-1)
        mat_list1[i] = Hp1
        mat_list2[i] = Hp2
        H1 = mat_list1[0]
        H2 = mat_list2[0]
        
        for j in range(N-2):
            H1 = tensor(H1, mat_list1[j+1])
            H2 = tensor(H2, mat_list2[j+1])
        
        if method == 'spectral_evol':
            psi = unit_evol(H1, psi, tau)   
            psi = unit_evol(H2, psi, tau) 
        
        elif method == 'cn':
            psi = crank_nicolson(H1, psi, tau, niter)
            psi = crank_nicolson(H2, psi, tau, niter)
        
        elif method == 'cn_LU':
            psi = crank_nicolson_LU(H1, psi, tau, niter)
            psi = crank_nicolson_LU(H2, psi, tau, niter)
            
        elif method == 'cn_LU_sp':
            psi = crank_nicolson_LU_sparse(H1, psi, tau, niter)
            psi = crank_nicolson_LU_sparse(H2, psi, tau, niter)
            
        else: 
            print("ERROR: no valid input method!")
            return None
        
    return fidelity(psi_ref,psi)

In [ ]:
def chain_CZ_gate_execution_fid(N, method, niter_list, ntimes=10):

    Omega   = 1
    frac_DO = 0.377371
    prod_Ot = 4.29268
    Delta = frac_DO * Omega
    tau = prod_Ot / Omega

    filename = "fid_"+str(method)+"_"+str(N)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)
    
    print("Method: ", method)
    for i in range(len(niter_list)):
        print("N: ", niter_list[i])
        
        state_first = 1
        state_last = 1
        psi_init = chain_init(N,state_first,state_last)
        
        fid = chain_CZ_gate_fid(psi_init,Omega,Delta,tau,method,niter_list[i]) 
    

        with open(filename, "a") as f:
            writer = csv.writer(f)
            writer.writerow((niter_list[i], fid))

In [ ]:
niter_list = np.logspace(np.log10(10),3,20,dtype=int)
N = 2

chain_CZ_gate_execution_fid(N, 'cn_LU', niter_list, ntimes=10)

## Phase analysis as a function of the number of iterations

In [ ]:
# Definition of phase function
def phase_func(z):
    
    # a + ib
    a = np.real(z)
    b = np.imag(z)
    
    if b==0:
        ph = 2*np.pi
    if (a>=0 and b>0): # I
        ph = np.arctan(b/a) 
    if (a<0 and b>0): # II
        ph = np.arctan(b/a) + np.pi
    if (a<0 and b<0): # III
        ph = np.arctan(b/a) + np.pi
    if (a>0 and b<0): # IV
        ph = 2*np.pi + np.arctan(b/a)
        
    return ph

In [ ]:
# Chain CZ-gate implementation
def chain_CZ_gate_phase(psi,Omega,Delta,tau,method='spectral_evol',niter=100):
    
    psi_ref = psi        
    N = int(np.log10(psi.shape[0])/(np.log10(3)))  # Number of qubits in the chain
    I = np.eye(3, dtype=data_type) # Define identity matrix
    Hp1 = hamiltonian(Omega,Delta) # First pulse hamiltonian
    Hp2 = hamiltonian(Omega * exp_xi(Delta,Omega,tau), Delta) # Second pulse hamiltonian  
    
    for i in range(N-1):
        
        start = time.time()
        mat_list1 = [I]*(N-1)
        mat_list2 = [I]*(N-1)
        mat_list1[i] = Hp1
        mat_list2[i] = Hp2
        H1 = mat_list1[0]
        H2 = mat_list2[0]
        
        for j in range(N-2):
            H1 = tensor(H1, mat_list1[j+1])
            H2 = tensor(H2, mat_list2[j+1])
        
        if method == 'spectral_evol':
            psi = unit_evol(H1, psi, tau)   
            psi = unit_evol(H2, psi, tau) 
        
        elif method == 'cn':
            psi = crank_nicolson(H1, psi, tau, niter)
            psi = crank_nicolson(H2, psi, tau, niter)
        
        elif method == 'cn_LU':
            psi = crank_nicolson_LU(H1, psi, tau, niter)
            psi = crank_nicolson_LU(H2, psi, tau, niter)
            
        elif method == 'cn_LU_sp':
            psi = crank_nicolson_LU_sparse(H1, psi, tau, niter)
            psi = crank_nicolson_LU_sparse(H2, psi, tau, niter)
            
        else: 
            print("ERROR: no valid input method!")
            return None
    
    return phase_func(np.conj(psi[4][0]))

In [ ]:
def chain_CZ_gate_execution_phase(N, method, niter_list, ntimes=10):

    Omega   = 1
    frac_DO = 0.377371
    prod_Ot = 4.29268
    Delta = frac_DO * Omega
    tau = prod_Ot / Omega

    filename = "phase_"+str(method)+"_"+str(N)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)
    
    print("Method: ", method)
    for i in range(len(niter_list)):
        print("N: ", niter_list[i])
        
        state_first = 1
        state_last = 1
        psi_init = chain_init(N,state_first,state_last)
        
        phase = chain_CZ_gate_phase(psi_init,Omega,Delta,tau,method,niter_list[i])

        with open(filename, "a") as f:
            writer = csv.writer(f)
            writer.writerow((niter_list[i],phase))

In [ ]:
niter_list = np.logspace(np.log10(10),3,20,dtype=int)
N = 2

chain_CZ_gate_execution_phase(N, 'cn_LU', niter_list, ntimes=10)